![UCA](http://caillau.perso.math.cnrs.fr/logo-uca.png)
## L2 MIASHS - ANDE
# TD 3 - Jeux

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/jbcaillau/ande/master?urlpath=lab/tree/td3/td3.ipynb)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jbcaillau/ande/blob/master/td3/td3.ipynb)

## Exercice 1
On considère le jeu classique de pierre-feuille-ciseaux, à deux joueurs, avec les règles suivantes :
* la pierre casse les ciseaux
* les ciseaux coupent le papier
* le papier recouvre la feuille

Le jeu est simultané, un joueur qui gagne marque un point, un joueur qui ne gagne pas (_i.e._ qui perd ou qui est à égalité) ne marque pas de point.

### 1.1
Mettre le jeu sous forme normale. Existe-t-il des équilibres ?

### 1.2
On suppose désormais que le joueur 2 arrive à deviner quand le joueur 1 s'apprête à jouer ciseaux (et uniquement ce coup). Proposer une nouvelle modélisation, et mettre le jeu sous forme extensive. Existe-t-il des équilibres ? Des stratégies dominées ou dominantes ?

(Source : F.-X. Dehon)